# Visualizacion del DataSet

En este Notebook se muestran algunos de los mecanismos más utilizados para la visualización de Datos.

### Abstract

NSL-KDD is a data set suggested to solve some of the inherent problems of the KDD'99 data set which are mentioned in [1]. Although, this new version of the KDD data set still suffers from some of the problems discussed by McHugh [2] and may not be a perfect representative of existing real networks, because of the lack of public data sets for network-based IDSs, we believe it still can be applied as an effective benchmark data set to help researchers compare different intrusion detection methods. Furthermore, the number of records in the NSL-KDD train and test sets are reasonable. This advantage makes it affordable to run the experiments on the complete set without the need to randomly select a small portion. Consequently, evaluation results of different research work will be consistent and comparable.

### Data Files

* <span style = "color:green">**KDDTrain+.ARFF** - The full NSL-KDD train set with binary labels in ARFF format </span>

* <span style = "color:green">**KDDTrain+.TXT**- The full NSL-KDD train set including attack-type labels and difficulty level in CSV format</span>

* KDDTrain+_20Percent.ARFF - A 20% subset of the KDDTrain+.arff file

* KDDTrain+_20Percent.TXT - A 20% subset of the KDDTrain+.txt file

* KDDTest+.ARFF - The full NSL-KDD test set with binary labels in ARFF format

* KDDTest+.TXT - The full NSL-KDD test set including attack-type labels and difficulty level in CSV format

* KDDTest-21.ARFF - A subset of the KDDTest+.arff file which does not include records with difficulty level of 21 out of 21

* KDDTest-21.TXT - A subset of the KDDTest+.txt file which does not include records with difficulty level of 21 out of 21

In [ ]:
# Lectura del DataSet mediante funciones de Python
with open('datasets/datasets/NSL-KDD/KDDTrain+.txt') as train_set:
    df = train_set.readlines()
df

In [ ]:
import pandas as pd
df = pd.read_csv("datasets/datasets/NSL-KDD/KDDTrain+.txt")
df

In [ ]:
import os
os.listdir('datasets/datasets/NSL-KDD')

In [ ]:
# instalar un nuevo paquetito en el kernel de Jupyter Notebook actual
# Para parsear ficheros .arff
import sys

!{sys.executable} -m pip install liac-arff

In [ ]:
# Lectura del DataSet que se encuentra en formato .arff
import arff
with open('datasets/datasets/NSL-KDD/KDDTrain+.arff', 'r') as train_set:
    df= arff.load(train_set)
df.keys()

In [ ]:
df["data"]

In [ ]:
df["attributes"]

In [ ]:
# Parsear los atributos y obtener unicamente los nombres
atributos = [attr[0] for attr in df ['attributes']]
atributos

In [ ]:
# Leer el DataSet con pandas y facilitar su manipulación
df = pd.DataFrame(df["data"], columns = atributos)
df

Lo ideal es contruir una función que permita leer el DataSet de manera más limpia. Este tipo de practicas son de gran utilidad, para que nuestro codigo en Jupyter Notebook sea más modular y pueda reutilizarse de manera más sencilla para futuros despliegues

In [ ]:
def load_kdd_dataset(data_path):
    """Lectura del DataSet NSL-KDD."""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
        attributes = [attr[0] for attr in dataset ['attributes']]
        return pd.DataFrame(dataset['data'], columns = attributes)

In [ ]:
load_kdd_dataset('datasets/datasets/NSL-KDD/KDDTrain+.arff')

# 2,- Funciones básicas de visualización de datos
* El proceso de visualización siempre debe realizarse sobre el training_set. Esto evita que nuestro entrenamiento genere intuiciones del test_set que se incorpore a nuestro modelo.
* Una buena practica es crear una copia del training-set y jugar con ella, de esta manera si se realiza transformaciones que dañen el training_set el original no se verá afectado

In [ ]:
# Lectura y copia del DataSet
df_orig = load_kdd_dataset("datasets/datasets/NSL-KDD/KDDTrain+.arff")
df= df_orig.copy()

In [ ]:
df.head(10)

In [ ]:
# Mostrar la información básica sobre el DataSet
df.info()

In [ ]:
# Mostrar las funciones estadisticas sobre el DataSet
df.describe()

In [ ]:
# Mostar los valores unicos que tiene un atributo determinado
df["protocol_type"].value_counts()

In [ ]:
df["class"].value_counts()

In [ ]:
# Mostrar lo valores de las caracteristicas como un histográma

%matplotlib inline
import matplotlib.pyplot as plt
df["protocol_type"].hist()

In [ ]:
# Representar gráficamnete la distribución de los atributos
df.hist(bins = 50, figsize=(20, 15))
plt.show()

# 3.- Funciones Avanzadas de Visualización de los Datos

Buscando correlaciones
* Se puede calcular el coeficiente de correlación para evaluar la correlación entre cada par de atributos
* El coeficiente de correlación, solo mide **correlaciones lineales**, esto quiere decir que si **x** va a hacia arriba, mediría si **y** va hacia abajo ó hacia arriba
* **Hay que intentar buscar correlaciones sobre todo con el atributo objetivo (el que se desea predecir), en este caso es class**

In [ ]:
# El atributo a class de nuestro DataSet tiene valores categóricos
df['class']

In [ ]:
# Transformar los valores del atributo class de categoricos a numéricos
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df["class"] = labelencoder.fit_transform(df["class"])
df

In [ ]:
# Mostrar la correlacion entre los atributos del DataSet

corr_matrix = df.corr(numeric_only = True)
corr_matrix ["class"].sort_values(ascending = False)

In [ ]:
# Mostrar la correlación lineal entre todos los atributos del DataSet
df.corr(numeric_only = True)

In [ ]:
# Representar graficamente la matriz de correlación
corr = df.corr(numeric_only = True)
fig, ax = plt.subplots( figsize = (8,8))
ax.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns);
plt.yticks(range(len(corr.columns)), corr.columns);

In [ ]:
# Representar graficamente las correlaciones 
from pandas.plotting import scatter_matrix
attributes = ['same_srv_rate', 'dst_host_srv_count', 'class', 'dst_host_same_srv_rate']

scatter_matrix (df[attributes], figsize=(12, 8))
plt.show()